In [1]:
# Summarize an article using the Chain of Density Prompting technique
# Article from: https://advanced-stack.com/resources/how-to-summarize-using-chain-of-density-prompting.html

In [2]:
import unicodedata
import PyPDF2
from llm_core.splitters import TokenSplitter

# Open the PDF file
with open('data/A Path Towards Autonomous Machine Intelligence.pdf', 'rb') as file:
    pdf_reader = PyPDF2.PdfReader(file)

    # Extract the text from the PDF
    pages = []
    for page in pdf_reader.pages:
        pages.append(page.extract_text())

    text = ''.join(pages)

def cleanup_unicode(text):
    corrected_chars = []
    for char in text:
        corrected_char = unicodedata.normalize("NFKC", char)
        corrected_chars.append(corrected_char)
    return "".join(corrected_chars)


text = cleanup_unicode(text)

In [3]:
print(text[:1000])

A Path Towards Autonomous Machine Intelligence
Version 0.9.2, 2022-06-27
Yann LeCun
Courant Institute of Mathematical Sciences, New York University yann@cs.nyu.edu
Meta - Fundamental AI Research yann@fb.com
June 27, 2022
Abstract
How could machines learn as eciently as humans and animals? How could ma-
chines learn to reason and plan? How could machines learn representations of percepts
and action plans at multiple levels of abstraction, enabling them to reason, predict,
and plan at multiple time horizons? This position paper proposes an architecture and
training paradigms with which to construct autonomous intelligent agents. It combines
concepts such as congurable predictive world model, behavior driven through intrinsic
motivation, and hierarchical joint embedding architectures trained with self-supervised
learning.
Keywords: Articial Intelligence, Machine Common Sense, Cognitive Architecture, Deep
Learning, Self-Supervised Learning, Energy-Based Model, World Models, Joint Embedd

In [26]:
splitter = TokenSplitter(chunk_size=3_000, chunk_overlap=0)
chunks = list(splitter.chunkify(text))

In [5]:
from dataclasses import dataclass
from llm_core.assistants import OpenAIAssistant, LLaMACPPAssistant


@dataclass
class DenseSummary:
    denser_summary: str
    missing_entities: list[str]


@dataclass
class DenserSummaryCollection:
  system_prompt = """
  You are an expert in writing rich and dense summaries in broad domains.
  """

  prompt = """
  Article:
  {article}
  ----

  You will generate increasingly concise, entity-dense summaries of the
  above Article.

  Repeat the following 2 steps 5 times.

  - Step 1: Identify 1-3 informative Entities from the Article
  which are missing from the previously generated summary and are the most
  relevant.

  - Step 2: Write a new, denser summary of identical length which covers
  every entity and detail from the previous summary plus the missing
  entities.

  A Missing Entity is:

  - Relevant: to the main story
  - Specific: descriptive yet concise (5 words or fewer)
  - Novel: not in the previous summary
  - Faithful: present in the Article
  - Anywhere: located anywhere in the Article

  Guidelines:
  - The first summary should be long (4-5 sentences, approx. 80 words) yet
  highly non-specific, containing little information beyond the entities
  marked as missing.

  - Use overly verbose language and fillers (e.g. "this article discusses")
  to reach approx. 80 words.

  - Make every word count: re-write the previous summary to improve flow and
  make space for additional entities.

  - Make space with fusion, compression, and removal of uninformative
  phrases like "the article discusses"

  - The summaries should become highly dense and concise yet
  self-contained, e.g., easily understood without the Article.

  - Missing entities can appear anywhere in the new summary.

  - Never drop entities from the previous summary. If space cannot be made,
  add fewer new entities.

  > Remember to use the exact same number of words for each summary.
  Answer in JSON.

  > The JSON in `summaries_per_step` should be a list (length 5) of
  dictionaries whose keys are "missing_entities" and "denser_summary".

  """

  summaries: list[DenseSummary]


  @classmethod
  def summarize(cls, article):
      with OpenAIAssistant(cls, model='gpt-4') as assistant:
          return assistant.process(article=article)

In [12]:
first_chunk = chunks[0]

In [7]:
summary_collection = DenserSummaryCollection.summarize(first_chunk)

In [8]:
print(len(summary_collection.summaries))

5


In [9]:
print(summary_collection.summaries[0].missing_entities)

['configurable predictive world models', 'intrinsic motivation', 'hierarchical joint embedding architectures']


In [10]:
print(summary_collection.summaries[0].denser_summary)

The article discusses the author's vision for creating autonomous intelligent agents that learn more like humans and animals. The author proposes an architecture that combines concepts such as configurable predictive world models, behavior driven through intrinsic motivation, and hierarchical joint embedding architectures trained with self-supervised learning. The author also identifies three main challenges that AI research must address today. The article is written in a way that appeals to readers from various backgrounds.


In [15]:
def single_pass_summary_generator(chunks):
  for chunk in chunks:
    chunk_summaries = DenserSummaryCollection.summarize(chunk)
    yield chunk_summaries.summaries[4].denser_summary


one_pass_summary = '\n'.join(single_pass_summary_generator(chunks))

In [18]:
splitter.compute_token_count(one_pass_summary)

722

In [20]:
summary_collection = DenserSummaryCollection.summarize(one_pass_summary)
final_summary = summary_collection.summaries[4].denser_summary

print(final_summary)

The paper proposes a vision for autonomous agents, integrating predictive world models, intrinsic motivation, hierarchical joint embedding architectures, including JEPA-1 and JEPA-2, Energy-Based Models (EBMs), Joint Embedding Architectures (JEAs), and Hierarchical JEPA. It scrutinizes a Model-Predictive Control process, emphasizing optimization strategies, and advocates for a multi-scale world model. It investigates machine learning, emphasizing architectural diagrams, and discusses the limitations of Large Language Models (LLMs) and reinforcement learning.


In [ ]:
# Using llama 3

In [27]:
@dataclass
class DenserSummaryCollection:
  system_prompt = """
  You are an expert in writing rich and dense summaries in broad domains.
  """

  prompt = """
  Article:
  {article}
  ----

  You will generate increasingly concise, entity-dense summaries of the
  above Article.

  Repeat the following 2 steps 5 times.

  - Step 1: Identify 1-3 informative Entities from the Article
  which are missing from the previously generated summary and are the most
  relevant.

  - Step 2: Write a new, denser summary of identical length which covers
  every entity and detail from the previous summary plus the missing
  entities.

  A Missing Entity is:

  - Relevant: to the main story
  - Specific: descriptive yet concise (5 words or fewer)
  - Novel: not in the previous summary
  - Faithful: present in the Article
  - Anywhere: located anywhere in the Article

  Guidelines:
  - The first summary should be long (4-5 sentences, approx. 80 words) yet
  highly non-specific, containing little information beyond the entities
  marked as missing.

  - Use overly verbose language and fillers (e.g. "this article discusses")
  to reach approx. 80 words.

  - Make every word count: re-write the previous summary to improve flow and
  make space for additional entities.

  - Make space with fusion, compression, and removal of uninformative
  phrases like "the article discusses"

  - The summaries should become highly dense and concise yet
  self-contained, e.g., easily understood without the Article.

  - Missing entities can appear anywhere in the new summary.

  - Never drop entities from the previous summary. If space cannot be made,
  add fewer new entities.

  > Remember to use the exact same number of words for each summary.
  Answer in JSON.

  > The JSON in `summaries_per_step` should be a list (length 5) of
  dictionaries whose keys are "missing_entities" and "denser_summary".

  """

  summaries: list[DenseSummary]


  @classmethod
  def summarize(cls, article):
      with LLaMACPPAssistant(cls, model='llama-3-q4') as assistant:
          return assistant.process(article=article)

In [ ]:
one_pass_summary = '\n'.join(single_pass_summary_generator(chunks))
print(one_pass_summary)

summary_collection = DenserSummaryCollection.summarize(one_pass_summary)
final_summary = summary_collection.summaries[4].denser_summary
print(final_summary)

In [29]:
final_summary

'An autonomous machine intelligence architecture is proposed combining predictive world models with hierarchical joint embedding architectures. JEPA combines perception, prediction, and action modules for hierarchical planning. The system learns from observation and predicts future outcomes using energy-based models.'